### 1) Creat a new database in PostgreSQL

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

PARALLEL = 4 # assuming a quad-core machine
ATTRIBUTE = "entity_phone"
#os.environ['SNORKELDB']
os.environ['SNORKELDBNAME'] = "location_extraction"
print os.environ['SNORKELDBNAME']
os.environ['SNORKELDB'] = 'postgresql://localhost:5432/' + os.environ['SNORKELDBNAME']
#from sqlalchemy import create_engine
#snorkeldb = create_engine('postgresql://localhost:5432/', isolation_level="AUTOCOMMIT")


sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/fonduer/memex/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
location_extraction


### Candidate Schema

In [4]:
from snorkel.contrib.fonduer import SnorkelSession

session = SnorkelSession()

In [5]:
import os
from snorkel.contrib.fonduer.models import candidate_subclass

Location_Extraction = candidate_subclass('location_extraction', ["location"])


### Running a Corpus preprocessor

In [6]:
from snorkel.contrib.fonduer import HTMLPreprocessor, OmniParser

docs_path = os.environ['SNORKELHOME'] + '/tutorials/fonduer/memex/data/profiles_chtap/'

doc_preprocessor = HTMLPreprocessor(docs_path)

In [7]:
corpus_parser = OmniParser(structural=True, lingual=True)
%time corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)

Clearing existing...
Running UDF...
CPU times: user 11 s, sys: 399 ms, total: 11.4 s
Wall time: 4min 17s


In [9]:
from snorkel.contrib.fonduer.models import Document, Phrase,Table

print "Documents:", session.query(Document).count()
print "Phrases:", session.query(Phrase).count()
print "Table", session.query(Table).count()

Documents: 364
Phrases: 76473
Table 891


In [10]:
docs = session.query(Document).order_by(Document.name).all()
ld   = len(docs)

train_docs = set()
dev_docs   = set()
test_docs  = set()
splits = (0.8, 0.9)
data = [(doc.name, doc) for doc in docs]
data.sort(key=lambda x: x[0])
for i, (doc_name, doc) in enumerate(data):
    if i < splits[0] * ld:
        train_docs.add(doc)
    elif i < splits[1] * ld:
        dev_docs.add(doc)
    else:
        test_docs.add(doc)
from pprint import pprint
#pprint([x.name for x in train_docs])
print "train:",len(train_docs)
print "dev:" ,len(dev_docs)
print "test:",len(test_docs)

train: 292
dev: 36
test: 36


In [16]:
train_docs.span[0]

AttributeError: 'set' object has no attribute 'span'

In [ ]:
from snorkel.matchers import *
